# Section 0
- Submission Date : 17/11/2024
- QPM 2024-2025: Assignment 3
- Members :
ESCATO Ugo  
HERVE Gauthier  
KAEPPELIN Teodor  
MEDDEB Malek  
CAO Yu
- Diversity :
1. Academic background : some did prepa, others went to univerisity,
2. Nationality : French, Chinese, Tunisian
3. Python : some very familiar with Python and machine learning, others not at all

# Section 1

## Question 1 of Assignment 3

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Tickers and date range
tickers = ['AAPL', 'MSFT', 'AMZN', 'NVDA', 'TSLA', 'META']
start_date = "2014-12-01"
end_date = "2022-12-31"

In [3]:
# Download data and process price data
dataset = yf.download(tickers, start=start_date, end=end_date)
dataset_prices = dataset['Adj Close']

[*********************100%***********************]  6 of 6 completed


In [4]:
dataset_prices.isnull().sum()

Ticker
AAPL    0
AMZN    0
META    0
MSFT    0
NVDA    0
TSLA    0
dtype: int64

In [5]:
# Convert to monthly frequency and compute returns
monthly_prices = dataset_prices.resample('M').last()
monthly_returns = monthly_prices.pct_change().dropna()

In [6]:
monthly_returns.head()

Ticker,AAPL,AMZN,META,MSFT,NVDA,TSLA
Date,,,,,,
2015-01-31 00:00:00+00:00,0.061424,0.142355,-0.027044,-0.130248,-0.042394,-0.084574
2015-02-28 00:00:00+00:00,0.100777,0.072293,0.040311,0.093120,0.153383,-0.001277
2015-03-31 00:00:00+00:00,-0.031372,-0.021202,0.041155,-0.072748,-0.051224,-0.071653
2015-04-30 00:00:00+00:00,0.005786,0.133513,-0.041961,0.196262,0.060679,0.197489
2015-05-31 00:00:00+00:00,0.045339,0.017663,0.005332,-0.030334,0.001461,0.109489


In [7]:
# Assume risk-free rate (rf) is 0
rf = 0

In [8]:
# Assume gamma is 5
gamma = 5

In [9]:
#index weights
weights=pd.DataFrame(index=monthly_returns.columns,columns=["Mean Returns Sample", "mu CAPM", "mu BL", "w Mkt", "w Markovitz MVP", "w BL"])
weights             

,Mean Returns Sample,mu CAPM,mu BL,w Mkt,w Markovitz MVP,w BL
Ticker,,,,,,
AAPL,NaN,NaN,NaN,NaN,NaN,NaN
AMZN,NaN,NaN,NaN,NaN,NaN,NaN
META,NaN,NaN,NaN,NaN,NaN,NaN
MSFT,NaN,NaN,NaN,NaN,NaN,NaN
NVDA,NaN,NaN,NaN,NaN,NaN,NaN
TSLA,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#w mkt
weights.index
for ticker in weights.index:
    stock = yf.Ticker(ticker)
    market_cap = stock.info['marketCap']  # Direct market cap field
    weights.loc[ticker]["w Mkt"]=market_cap
weights["w Mkt"]/=weights["w Mkt"].sum() # normalized
weights

,Mean Returns Sample,mu CAPM,mu BL,w Mkt,w Markovitz MVP,w BL
Ticker,,,,,,
AAPL,NaN,NaN,NaN,0.234103,NaN,NaN
AMZN,NaN,NaN,NaN,0.146644,NaN,NaN
META,NaN,NaN,NaN,0.096281,NaN,NaN
MSFT,NaN,NaN,NaN,0.212381,NaN,NaN
NVDA,NaN,NaN,NaN,0.239727,NaN,NaN
TSLA,NaN,NaN,NaN,0.070865,NaN,NaN


In [11]:
#return sample
weights["Mean Returns Sample"]=monthly_returns.mean()
weights

,Mean Returns Sample,mu CAPM,mu BL,w Mkt,w Markovitz MVP,w BL
Ticker,,,,,,
AAPL,0.020842,NaN,NaN,0.234103,NaN,NaN
AMZN,0.021847,NaN,NaN,0.146644,NaN,NaN
META,0.009133,NaN,NaN,0.096281,NaN,NaN
MSFT,0.020731,NaN,NaN,0.212381,NaN,NaN
NVDA,0.044966,NaN,NaN,0.239727,NaN,NaN
TSLA,0.036711,NaN,NaN,0.070865,NaN,NaN


### Question 1.1

In [12]:
expected_returns = monthly_returns.mean() 
cov_matrix = monthly_returns.cov()         

In [13]:
def MVP(gamma, rf, V, mu):
    """
    Calculate the weights of the mean-variance portfolio (MVP), 
    the weight in the risk-free asset, and the weights of the tangency portfolio.

    Parameters:
    - gamma (float): Investor's risk aversion parameter
    - rf (float): Gross risk-free rate
    - V (ndarray): Variance-covariance matrix of returns
    - mu (ndarray): Column vector of gross mean returns

    Returns:
    - w_mvp (ndarray): Weights of the mean-variance portfolio (MVP)
    - w_riskFree (float): Weight in the risk-free asset
    - w_tangency (ndarray): Weights of the tangency portfolio
    """
    # Ensure mu is a numpy array with appropriate shape
    mu = np.array(mu).reshape(-1, 1)
    
    # Column vector of ones
    ones = np.ones((mu.shape[0], 1))
    
    # Inverse of the variance-covariance matrix
    Vinverse = np.linalg.inv(V)
    
    # Weights for the mean-variance portfolio (MVP)
    w_mvp = (1 / gamma) * Vinverse @ (mu - rf * ones)
    
    # Weight in the risk-free asset
    w_riskFree = 1 - ones.T @ w_mvp
    
    # Weights of the tangency portfolio
    w_tangency = w_mvp / (ones.T @ w_mvp)
    
    return w_mvp, w_riskFree, w_tangency

In [14]:
w_mvp, w_riskFree, w_tangency = MVP(gamma, rf, cov_matrix, expected_returns)

In [15]:
weights["w Markovitz MVP"]=w_tangency
weights

,Mean Returns Sample,mu CAPM,mu BL,w Mkt,w Markovitz MVP,w BL
Ticker,,,,,,
AAPL,0.020842,NaN,NaN,0.234103,0.003690,NaN
AMZN,0.021847,NaN,NaN,0.146644,0.058800,NaN
META,0.009133,NaN,NaN,0.096281,-0.241624,NaN
MSFT,0.020731,NaN,NaN,0.212381,0.683057,NaN
NVDA,0.044966,NaN,NaN,0.239727,0.406413,NaN
TSLA,0.036711,NaN,NaN,0.070865,0.089664,NaN


### Question 1.2

In [16]:
# Cap-weighted portfolio
weights["w Mkt"]

Ticker
AAPL    0.234103
AMZN    0.146644
META    0.096281
MSFT    0.212381
NVDA    0.239727
TSLA    0.070865
Name: w Mkt, dtype: object

### Question 1.3

In [17]:
gamma_market = expected_returns@weights["w Mkt"]/(weights["w Mkt"].T @ cov_matrix @ weights["w Mkt"])

In [18]:
ones = np.ones(len(cov_matrix))

In [19]:
implied_expected_returns = gamma_market * cov_matrix @ weights["w Mkt"] + rf*ones
weights["mu CAPM"] = implied_expected_returns
weights

,Mean Returns Sample,mu CAPM,mu BL,w Mkt,w Markovitz MVP,w BL
Ticker,,,,,,
AAPL,0.020842,0.023796,NaN,0.234103,0.003690,NaN
AMZN,0.021847,0.024048,NaN,0.146644,0.058800,NaN
META,0.009133,0.018505,NaN,0.096281,-0.241624,NaN
MSFT,0.020731,0.01805,NaN,0.212381,0.683057,NaN
NVDA,0.044966,0.038959,NaN,0.239727,0.406413,NaN
TSLA,0.036711,0.03802,NaN,0.070865,0.089664,NaN


### Question 1.4

In [20]:
q = (1 + np.array([0.1,0.05,0.02,0.01]))**(1/12) -1 # convert into monthly returns

In [21]:
P = np.array([
    [1, 0, 0, 0, 0, 0],   # View on AAPL (10% excess return)
    [0, 0, 0, 1, 0, 0],   # View on MSFT (5% excess return)   
    [0, 0, 0, 0, 1, -1],  # View: NVDA outperforms TSLA by 2%
    [0, 0, 1, 0, 0, -1]   # View: TSLA underperforms META by 1%
])

In [22]:
tau = 1/len(monthly_returns)

In [23]:
omega = np.zeros((P.shape[0], P.shape[0]))  # Create omega matrix with zeros

# Loop through each view to calculate omega
for k in range(P.shape[0]):
    omega[k, k] = np.dot(np.dot(P[k], tau * cov_matrix), P[k].T)

print("Omega (Covariance Matrix for Views):\n", omega)

Omega (Covariance Matrix for Views):
 [[7.36568271e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 4.39399123e-05 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 3.68662830e-04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 3.40246862e-04]]


### Question 1.5

In [24]:
# Compute Black-Litterman expected returns (mu_BL)
term1 = np.linalg.inv(tau * cov_matrix)
term2 = P.T @ np.linalg.inv(omega) @ P
term3 = np.linalg.inv(tau * cov_matrix) @ implied_expected_returns
term4 = P.T @ np.linalg.inv(omega) @ q

mu_BL = np.linalg.inv(term1 + term2) @ (term3 + term4)

print("Black-Litterman Expected Returns (mu BL):\n", mu_BL)

Black-Litterman Expected Returns (mu BL):
 [0.0132698720649901 0.015406679981755288 0.013565410728654225
 0.009581513967482202 0.025673125737180075 0.020784819632785567]


In [25]:
weights["mu BL"]=mu_BL
weights

,Mean Returns Sample,mu CAPM,mu BL,w Mkt,w Markovitz MVP,w BL
Ticker,,,,,,
AAPL,0.020842,0.023796,0.01327,0.234103,0.003690,NaN
AMZN,0.021847,0.024048,0.015407,0.146644,0.058800,NaN
META,0.009133,0.018505,0.013565,0.096281,-0.241624,NaN
MSFT,0.020731,0.01805,0.009582,0.212381,0.683057,NaN
NVDA,0.044966,0.038959,0.025673,0.239727,0.406413,NaN
TSLA,0.036711,0.03802,0.020785,0.070865,0.089664,NaN


In [26]:
sigma_BL = cov_matrix + np.linalg.inv((term1)+ term2)

print("Black-Litterman Covariance Matrix (sigma BL):\n", sigma_BL)

Black-Litterman Covariance Matrix (sigma BL):
 Ticker      AAPL      AMZN      META      MSFT      NVDA      TSLA
Ticker                                                            
AAPL    0.007104  0.004218  0.003164  0.003214  0.006413  0.008014
AMZN    0.004218  0.008554  0.004071  0.003722  0.006345  0.006960
META    0.003164  0.004071  0.009030  0.002833  0.004397  0.004590
MSFT    0.003214  0.003722  0.002833  0.004238  0.005199  0.005066
NVDA    0.006413  0.006345  0.004397  0.005199  0.017973  0.007689
TSLA    0.008014  0.006960  0.004590  0.005066  0.007689  0.032953


### Question 1.6

In [27]:
weights_BL = (1/gamma_market) * np.linalg.inv(sigma_BL) @ mu_BL
weights_BL = weights_BL/sum(weights_BL) # normalize
weights["w BL"]=weights_BL

In [28]:
weights

,Mean Returns Sample,mu CAPM,mu BL,w Mkt,w Markovitz MVP,w BL
Ticker,,,,,,
AAPL,0.020842,0.023796,0.01327,0.234103,0.003690,0.124371
AMZN,0.021847,0.024048,0.015407,0.146644,0.058800,0.270819
META,0.009133,0.018505,0.013565,0.096281,-0.241624,0.279887
MSFT,0.020731,0.01805,0.009582,0.212381,0.683057,-0.146596
NVDA,0.044966,0.038959,0.025673,0.239727,0.406413,0.405196
TSLA,0.036711,0.03802,0.020785,0.070865,0.089664,0.066323


### Question 1.7

In [29]:
print(f"Markowitz MVP weights, {weights['w Markovitz MVP']}")
print(f"\nCap-weighted (market) portfolio weights, {weights['w Mkt']}")
print(f"\nBL MVP weights, {weights['w BL']}")

Markowitz MVP weights, Ticker
AAPL    0.003690
AMZN    0.058800
META   -0.241624
MSFT    0.683057
NVDA    0.406413
TSLA    0.089664
Name: w Markovitz MVP, dtype: float64

Cap-weighted (market) portfolio weights, Ticker
AAPL    0.234103
AMZN    0.146644
META    0.096281
MSFT    0.212381
NVDA    0.239727
TSLA    0.070865
Name: w Mkt, dtype: object

BL MVP weights, Ticker
AAPL    0.124371
AMZN    0.270819
META    0.279887
MSFT   -0.146596
NVDA    0.405196
TSLA    0.066323
Name: w BL, dtype: object


### Conclusion

The portfolio weights comparison between three methodologies-Markowitz Mean-Variance Portfolio, cap-weighted, or market portfolio, and Black-Litterman-reveals different allocation strategies driven by their optimization frameworks and assumptions that underpin them:

1. **Markowitz Mean-Variance Portfolio Weights**:
    - The portfolio has a heavy allocation to MSFT at 68.3% and NVDA at 40.6%, while it assigns a negative weight to META at 24.2%, perhaps reflective of its unfavorable expected return or risk-adjusted trade-offs.
    - This approach seeks to maximize return for a given level of risk, relying on historical estimates of returns and covariances without incorporating market equilibrium or investor views.


2. **Cap-Weighted (Market) Portfolio Weights**:
    - The portfolio weights are passive and reflect each asset's share of total market capitalization. The largest portfolio weighting components include NVDA at 23.9%, AAPL at 23.4%, and AMZN at 14.7% percent.
    - It does not actively seek to make adjustments for risk-return trade-offs but acts instead as a benchmark of market performance.


3. **Black-Litterman (BL) MVP Weights**:
    - The BL model offers a method for incorporating investor views into market equilibrium weights to get the optimal portfolio that balances subjective investor expectations with market information.
    - META is highly weighted at 27.9%, NVDA with 40.5%, while MSFT has a negative weighting of -14.7%, reflecting investor views.
    - The BL framework corrects the pure historically-based methods shortcomings with regard to giving an allocation that is more adaptive.
    
Ultimately, the choice of methodology should align with the investor's objectives, risk tolerance, and confidence in their views versus historical or market data.